MonthEnd Automation Production

In [1]:
import requests
import pandas as pd
from io import BytesIO
import msal
import os
import win32com.client as win32
import pythoncom
from win32com.client import Dispatch
import time

In [ ]:
# Define tenant-specific values (replace these with your actual values)
tenant_id = "your_tenant_secret"
client_id = "your_client_id"
client_secret = "your_client_secret"

In [3]:
# Set up MSAL authentication
app = msal.ConfidentialClientApplication(
    client_id=client_id,
    client_credential=client_secret,
    authority=f"https://login.microsoftonline.com/{tenant_id}",
)

In [4]:
# Function to acquire access token
def get_access_token():
    scope = ["https://graph.microsoft.com/.default"]
    token_response = app.acquire_token_for_client(scopes=scope)
    access_token = token_response.get("access_token")
    if access_token:
        print("Access token acquired successfully.")
        return access_token
    else:
        raise Exception("Failed to acquire access token.")

# Get the token
access_token = get_access_token()

# Set headers with the access token for future requests
headers = {
    "Authorization": f"Bearer {access_token}"
}

Access token acquired successfully.


In [ ]:
#define variable direction of sharepoint *CHANGE IT* with your user of your local
url_people = 'your_name'

# From SHarepoint (takes longer but takes it directly live from sharepoint)
excel_files = {
    'dynamicPL': fr'C:\Users\{url_people}\workfile.xlsx',
    'biweekly': fr'C:\Users\{url_people}\Biweekly Report.xlsx',
}


In [ ]:
import time
import pythoncom
from win32com.client import Dispatch

# Function to load Excel files from SharePoint
def load_excel_from_sharepoint(file_url):
    response = requests.get(file_url, headers=headers)
    if response.status_code == 200:
        excel_data = BytesIO(response.content)
        df = pd.read_excel(excel_data, sheet_name='Master List')
        print(f"Mapping Excel file loaded with {len(df)} rows.")
        return df
    else:
        raise Exception(f"Failed to load Excel file: {response.status_code}, {response.text}")

# Load the mapping file from SharePoint
# Load the mapping file from SharePoint
mapping_file_id = 'your_mapping_id' 
drive_id = 'your_driver_id' 
mapping_file_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{mapping_file_id}/content"

##################################################
# Load the mapping file into `mapping_df`
try:
    mapping_df = load_excel_from_sharepoint(mapping_file_url)
    print(f"Mapping file loaded successfully with {len(mapping_df)} rows.")
except Exception as e:
    print(f"Error loading mapping file: {e}")
    mapping_df = None

# Ensure the mapping_df is loaded before continuing
if mapping_df is not None:
    # Filter the DataFrame to only include rows with valid Named Ranges
    valid_mapping_df = mapping_df[mapping_df['Named Range'].notna() & (mapping_df['Named Range'] != '')]

    # Convert Slide IDs to a numeric format
    valid_mapping_df['Slide ID Numeric'] = valid_mapping_df['Slide ID'].apply(lambda x: int(x.replace('Slide', '').strip()) if pd.notna(x) else None)

    ppt_path = fr"C:\Users\{url_people}\Downloads\Template.pptx"

    # Open PowerPoint
    try:
        pptApp = Dispatch('PowerPoint.Application')
        pptPresentation = pptApp.Presentations.Open(ppt_path)

        # Iterate through all the slides in the PowerPoint presentation
        for slide in pptPresentation.Slides:
            slide_id = slide.SlideID
            print(f"Checking Slide ID: {slide_id}, Slide Index: {slide.SlideIndex}")

            # Check if this slide matches any row in the valid_mapping_df
            for index, row in valid_mapping_df.iterrows():
                if slide_id == row['Slide ID Numeric']:

                    print(f"Match found for Slide ID {slide_id}: Named Range {row['Named Range']} in file {row['File ID']}")

                    # Retrieve the Excel file path from the excel_files dictionary
                    excel_file = excel_files.get(row['File ID'])
                    if excel_file is None:
                        print(f"Excel file for File ID '{row['File ID']}' not found.")
                        continue

                    # Open the Excel file and wait for it to fully load
                    excelApp = Dispatch("Excel.Application")
                    excelApp.Visible = False  # Keep it hidden
                    excelApp.DisplayAlerts = False  # Disable alerts
                    excelApp.AskToUpdateLinks = False  # Prevent the update links prompt
                    excelApp.AlertBeforeOverwriting = False  # Prevent overwrite prompts

                    try:
                        excelWorkbook = excelApp.Workbooks.Open(excel_file, UpdateLinks=0)  # Disable link updates
                        time.sleep(1)  # Small delay to ensure the workbook is fully loaded

                        try: 
                            # Get size and position of the existing image (if any)
                            existing_image = None
                            for shape in slide.Shapes:
                                if shape.Type == 13:  # Type 13 is Picture
                                    existing_image = shape
                                    break

                            if existing_image:
                                old_left = existing_image.Left
                                old_top = existing_image.Top
                                old_width = existing_image.Width
                                old_height = existing_image.Height

                                # Remove the old image
                                existing_image.Delete()

                            # Check if the named range exists in the workbook
                            if row['Named Range'] in [name.Name for name in excelWorkbook.Names]:
                                named_range = excelWorkbook.Names(row['Named Range']).RefersToRange
                                named_range.CopyPicture(Format=2)  # Format 2 = Picture

                                # Paste the copied image onto the slide
                                pasted_shape = slide.Shapes.Paste()
                                new_image = pasted_shape.Item(1)
                                print(f"Pasted named range {row['Named Range']} on Slide ID {slide_id}")

                                # Send the new image to the back (behind all other content)
                                new_image.ZOrder(1)  # 1 corresponds to msoSendToBack (send to back)

                                # Unlock aspect ratio to resize freely
                                new_image.LockAspectRatio = False

                                # Set the new image to the same size and position as the old image
                                new_image.Left = old_left
                                new_image.Top = old_top
                                new_image.Width = old_width
                                new_image.Height = old_height
                                print(f"Resized and repositioned the image on Slide ID {slide_id} to match the previous image.")
                            else:
                                print(f"Named range '{row['Named Range']}' not found in file '{row['File ID']}'")
                                continue

                        except Exception as e:
                            print(f"Error accessing named range '{row['Named Range']}' in file '{row['File ID']}': {e}")

                    except pythoncom.com_error as com_err:
                        print(f"COM Error while opening Excel file '{excel_file}': {com_err}")

                    finally:
                        excelWorkbook.Close(SaveChanges=False)
                        excelApp.Quit()

    except pythoncom.com_error as e:
        print(f"COM Error: {e}")
else:
    print("No valid mapping data to process.")

Other Important Codes

In [ ]:
#List of Slide ID into a CSV
    #This will allow you to identify the Slide ID of the power Point. 

import csv
import os
import win32com.client as win32

# Open PowerPoint application
pptApp = win32.Dispatch("PowerPoint.Application")
pptApp.Visible = True

# Load the existing PowerPoint template
ppt_template_path = r"C:\Users\Template.pptx"
pptPresentation = pptApp.Presentations.Open(ppt_template_path)

# Print confirmation that PowerPoint is loaded and display the number of slides
print(f"PowerPoint loaded with {pptPresentation.Slides.Count} slides.")

# Create a dictionary to map Slide ID to Slide Index
slide_id_lookup = {slide.SlideID: slide.SlideIndex for slide in pptPresentation.Slides}

# Create a CSV file to save the Slide ID to Slide Index mapping
csv_file_path = r"C:\Users\slide_id_mapping.csv"


# Write to CSV
with open(csv_file_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Slide ID", "Slide Index"])  # Write header
    for slide_id, slide_index in slide_id_lookup.items():
        csv_writer.writerow([slide_id, slide_index])  # Write each row
    
print(f"CSV file created: {csv_file_path}")

# Optionally close the PowerPoint file
pptPresentation.Close()
pptApp.Quit()
